In [ ]:
!pip install transformers 

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
     |████████████████████████████████| 596 kB 46.5 MB/s 
     |████████████████████████████████| 6.6 MB 36.6 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, TFAutoModel, AutoConfig, pipeline, AutoModel
import re
import io
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score, accuracy_score,precision_score,recall_score
import os
from google.colab import drive
seed =45
np.random.seed(seed)
tf.random.set_seed(seed)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tf.distribute.cluster_resolver.TPUClusterResolver.connect())
drive.mount('/gdrive')

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.26.174.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.26.174.26:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Mounted at /gdrive


In [ ]:
model_type="UBC-NLP/ARBERT"
language="Ara"
data_dir="/content"
train_file="ara_train.csv"
dev_file="ara_dev.csv"
model_dir="/content"
model_file=f"{language}-Mono-Model.h5"
train_path=os.path.join(data_dir,train_file)
dev_path=os.path.join(data_dir,dev_file)
model_path=os.path.join(model_dir,model_file)

In [ ]:
train_data=pd.read_csv(train_path)
valid_data=pd.read_csv(dev_path)

In [ ]:
def remove_non_arabic_tokens(data,column):
    data[column]=data[column].apply(lambda text:re.sub(r'[^0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD.0-9]+',' ', text))
    return data

def remove_white_spaces(data,column):
    data[column]=data[column].apply(lambda text:re.sub(' +', ' ', text))
    data[column]=data[column].apply(lambda text:text.strip())
    return data

def transform_hashtags(data,column):
    def transform(text):
        new_text=[]
        for word in text.strip().split(" "):
            if "#" in word:
                word=word.replace("#","")
                word=word.split("_")
                word=" ".join(word)
            new_text.append(word)
        return " ".join(new_text)
    data[column]=data[column].apply(lambda text:transform(text))
    return data


def text_preprocessing_pipeline(data,target):
    data=transform_hashtags(data,target)
    data=remove_non_arabic_tokens(data,target) 
    data=remove_white_spaces(data,target)
    return data
           
#train_data=train_data.append(text_preprocessing_pipeline(train_data,"tweet_text")).sample(frac=1).reset_index(drop=True)
# valid_data=text_preprocessing_pipeline(valid_data,"tweet_text")

In [ ]:
def balanced_batch(data):
  harmful=data.loc[data["class_label"]==1]
  norm_count=int(len(harmful)*1.5)
  normal=data.loc[data["class_label"]==0].sample(n=norm_count)
  data=harmful.append(normal).sample(frac=1).reset_index(drop=True)
  return data

train_data=balanced_batch(train_data)

In [ ]:
def gen_data(data,tokenizer_type=model_type):
    data_dict={}
    data_dict["harmful_label"]=tf.convert_to_tensor(data["class_label"].values.astype(int))
    data_dict["text"]=data["tweet_text"].values.astype("str")
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)
    input_ids, input_masks = [],[]
    for sentence in data_dict["text"]:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,max_length=326,truncation=True, padding='max_length',return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
    data_dict["input_ids"]=tf.convert_to_tensor(input_ids) 
    data_dict["input_masks"]=tf.convert_to_tensor(input_masks)  
    return data_dict

train=gen_data(train_data)
valid=gen_data(valid_data)

Downloading:   0%|          | 0.00/374 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/701 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# def create_mono_model(transformer):
#     input_ids= tf.keras.layers.Input(shape=(326,), dtype='int32')
#     input_masks = tf.keras.layers.Input(shape=(326,), dtype='int32')
#     embedding_layer=transformer(input_ids, attention_mask=input_masks)[0]
#     #embedding_layer= tf.keras.layers.SpatialDropout1D(0.2)(embedding_layer)
    
#     harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True, dropout=0.2))(embedding_layer)
#     harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True, dropout=0.1))(harmful)
#     harmful = tf.keras.layers.Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(harmful)
#     avg_pool_harmful = tf.keras.layers.GlobalAveragePooling1D()(harmful)
#     max_pool_harmful = tf.keras.layers.GlobalMaxPooling1D()(harmful)
#     harmful = tf.keras.layers.concatenate([avg_pool_harmful, max_pool_harmful])
#     harmful = tf.keras.layers.Dropout(0.1)(harmful)
#     output_harmful=tf.keras.layers.Dense(1, activation="sigmoid",name="output_harmful")(harmful)

#     model = tf.keras.Model(inputs=[input_ids, input_masks], outputs =output_harmful)
    
#     return model

In [ ]:
def create_mono_model(transformer):
    input_ids= tf.keras.layers.Input(shape=(326,), dtype='int32')
    input_masks = tf.keras.layers.Input(shape=(326,), dtype='int32')
    embedding_layer =transformer(input_ids, attention_mask=input_masks)[0]
    #embedding_layer= tf.keras.layers.SpatialDropout1D(0.2)(embedding_layer)
    
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.5))(embedding_layer)
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, dropout=0.4))(harmful)
    harmful=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True, dropout=0.3))(harmful)
    harmful = tf.keras.layers.Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(harmful)
    harmful = tf.keras.layers.Dropout(0.2)(harmful)
    avg_pool_harmful = tf.keras.layers.GlobalAveragePooling1D()(harmful)
    max_pool_harmful = tf.keras.layers.GlobalMaxPooling1D()(harmful)
    harmful = tf.keras.layers.concatenate([avg_pool_harmful, max_pool_harmful])
    harmful = tf.keras.layers.Dropout(0.05)(harmful)
    output_harmful=tf.keras.layers.Dense(1, activation="sigmoid",name="output_harmful")(harmful)

    model = tf.keras.Model(inputs=[input_ids, input_masks], outputs =output_harmful)
    
    return model

In [ ]:
def build_mono_model(compile=False,show_summary=False,pretrained_model=model_type):
    with tpu_strategy.scope():
      config =AutoConfig.from_pretrained(pretrained_model,dropout=0.2,seed=3,attention_dropout=0.2,output_hidden_states = True)
      transformer= TFAutoModel.from_pretrained(pretrained_model,config=config)
      model =create_mono_model(transformer)
      if compile : 
          model.compile(loss={"output_harmful":"binary_crossentropy"},
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                      metrics={"output_harmful":"accuracy"}
                     )
      if show_summary:
        model.summary()
      return model

model=build_mono_model()

def train_model(model,train,valid,batch_size=64,epochs_frozen=4,epochs_unfrozen=10):
      
    checkpoint_filepath = data_dir

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_filepath, 
        monitor='val_loss', 
        save_weights_only=True, 
        save_best_only=True, 
        save_freq='epoch', 
        options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'))
    
    for layer in model.layers[:3]:
        layer.trainable = False
   
    with tpu_strategy.scope():
      model.compile(loss={"output_harmful":"binary_crossentropy"},
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                      metrics={"output_harmful":"accuracy"}
                     )
        
    model.fit([train["input_ids"], train["input_masks"]],train["harmful_label"],batch_size=batch_size,epochs=epochs_frozen,validation_data=([valid["input_ids"],valid["input_masks"]],valid["harmful_label"]), callbacks=[model_checkpoint_callback])
   
    for layer in model.layers[:3]:
        layer.trainable = True

    with tpu_strategy.scope():
      model.compile(loss={"output_harmful":"binary_crossentropy"}, 
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
                      metrics={"output_harmful":"accuracy"})
    
    model.fit([train["input_ids"], train["input_masks"]],train["harmful_label"],batch_size=batch_size,epochs=epochs_unfrozen,validation_data=([valid["input_ids"],valid["input_masks"]],valid["harmful_label"]), callbacks=[model_checkpoint_callback])

train_model(model,train,valid)

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at UBC-NLP/ARBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/4
29/29 [==============================] - 106s 2s/step - loss: 0.5498 - accuracy: 0.7182 - val_loss: 0.5462 - val_accuracy: 0.7586
Epoch 2/4
29/29 [==============================] - 15s 538ms/step - loss: 0.4377 - accuracy: 0.7913 - val_loss: 0.5258 - val_accuracy: 0.7897
Epoch 3/4
29/29 [==============================] - 15s 530ms/step - loss: 0.3927 - accuracy: 0.8238 - val_loss: 0.4201 - val_accuracy: 0.8270
Epoch 4/4
29/29 [==============================] - 15s 535ms/step - loss: 0.3769 - accuracy: 0.8282 - val_loss: 0.3756 - val_accuracy: 0.8561
Epoch 1/10


29/29 [==============================] - 235s 4s/step - loss: 0.3969 - accuracy: 0.8184 - val_loss: 0.3432 - val_accuracy: 0.8803
Epoch 2/10
29/29 [==============================] - 17s 588ms/step - loss: 0.2478 - accuracy: 0.9035 - val_loss: 0.3809 - val_accuracy: 0.8753
Epoch 3/10
29/29 [==============================] - 17s 583ms/step - loss: 0.1601 - accuracy: 0.9474 - val_loss: 0.3727 - val_accuracy: 0.8763
Epoch 4/10
29/29 [==============================] - 17s 585ms/step - loss: 0.0893 - accuracy: 0.9696 - val_loss: 0.4056 - val_accuracy: 0.8863
Epoch 5/10
29/29 [==============================] - 17s 586ms/step - loss: 0.0486 - accuracy: 0.9886 - val_loss: 0.6200 - val_accuracy: 0.8531
Epoch 6/10
29/29 [==============================] - 17s 585ms/step - loss: 0.0336 - accuracy: 0.9930 - val_loss: 0.4693 - val_accuracy: 0.8873
Epoch 7/10
29/29 [==============================] - 17s 590ms/step - loss: 0.0223 - accuracy: 0.9951 - val_loss: 0.4950 - val_accuracy: 0.8843
Epoch 8/10
2

In [ ]:
def report_gen(predictions,labels,avg="binary"):
    report={
    "F1_macro":f1_score(predictions,labels,average=avg),
    "Accuracy":accuracy_score(predictions,labels),
    "Precision_macro":precision_score(predictions,labels),
    "Recall_macro":recall_score(predictions,labels)
    }
    return report

def eval_task_harmful(predictions_harmful,labels_harmful):
  predictions_harmful=[int(i>0.9) for i in predictions_harmful]
  return report_gen(predictions_harmful,labels_harmful),predictions_harmful

def eval_model(model,valid):
    predictions_harmful=model.predict([valid["input_ids"],valid["input_masks"]])
    report_harmful,predictions_harmful=eval_task_harmful(predictions_harmful,valid["harmful_label"])
    print(report_harmful)
    return predictions_harmful

predictions=eval_model(model,valid)

{'F1_macro': 0.7445255474452555, 'Accuracy': 0.8943661971830986, 'Precision_macro': 0.8095238095238095, 'Recall_macro': 0.6891891891891891}


In [ ]:
# 256 128 64 / 64 

In [ ]:
#pd.DataFrame(predictions).to_csv("/content/predictions.csv",index=False, header=None)